# PyVOQC Tutorial (PLanQC 2021)

This tutorial introduces PyVOQC, the Python bindings for the VOQC optimizer (available at [inQWIRE/pyvoqc](https://github.com/inQWIRE/pyvoqc)). We first show how to use PyVOQC as a pass in Qiskit (our recommended method), and then show how to call PyVOQC functions directly.

## Preliminaries

To run this tutorial: 
1. Install our OCaml package with `opam install voqc` (requires opam)
2. Run `./install.sh` in the pyvoqc directory

For more details and troubleshooting, see the [README](https://github.com/inQWIRE/pyvoqc/REAMDE.md) in the pyvoqc repository.

## Running PyVOQC as a Qiskit Pass

Using our `voqc_pass` wrapper, VOQC can be called just like any other optimization pass in [IBM's Qiskit framework](https://qiskit.org/documentation/getting_started.html). This allows us to take advantage of Qiskit's utilities for quantum programming, such as the ability to build and print circuits.

To use VOQC, simply append `QiskitVOQC([opt list])` to a Qiskit `Pass Manager` where `opt list` is an optional argument specifying one or more of the transformations in VOQC. `QiskitVOQC()` with no arguments will run all available optimizations.

In [1]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import QiskitVOQC
from qiskit.transpiler import PassManager

# create a circuit using Qiskit's interface
circ = QuantumCircuit(2)
circ.x(0)
circ.t(0)
circ.t(1)
circ.cz(0, 1)
circ.t(0)
circ.tdg(1)
print("Before Optimization:")
print(circ)

Before Optimization:
     ┌───┐┌───┐    ┌───┐ 
q_0: ┤ X ├┤ T ├─■──┤ T ├─
     ├───┤└───┘ │ ┌┴───┴┐
q_1: ┤ T ├──────■─┤ TDG ├
     └───┘        └─────┘


In [3]:
# create a Qiskit PassManager
pm = PassManager()

# decompose CZ gate
pm.append(QiskitVOQC(["decompose_to_cnot"]))
new_circ = pm.run(circ)
print("After 'decompose_to_cnot':")
print(new_circ)

After 'decompose_to_cnot':
     ┌───┐┌───┐     ┌───┐       
q_0: ┤ X ├┤ T ├──■──┤ T ├───────
     ├───┤├───┤┌─┴─┐├───┤┌─────┐
q_1: ┤ T ├┤ H ├┤ X ├┤ H ├┤ TDG ├
     └───┘└───┘└───┘└───┘└─────┘


In [4]:
# run optimizations from Nam et al.
pm.append(QiskitVOQC(["optimize_nam"]))
new_circ = pm.run(circ)
print("After 'optimize_nam':")
print(new_circ)

After 'optimize_nam':
               ┌─────┐┌───┐
q_0: ───────■──┤ SDG ├┤ X ├
     ┌───┐┌─┴─┐└┬───┬┘├───┤
q_1: ┤ H ├┤ X ├─┤ H ├─┤ Z ├
     └───┘└───┘ └───┘ └───┘


In [5]:
# run IBM gate merging
pm.append(QiskitVOQC(["optimize_ibm"]))
new_circ = pm.run(circ)
print("After 'optimize_ibm':")
print(new_circ)

After 'optimize_ibm':
                     ┌─────────────┐
q_0: ─────────────■──┤ U3(π,0,π/2) ├
     ┌─────────┐┌─┴─┐└─┬─────────┬─┘
q_1: ┤ U2(0,π) ├┤ X ├──┤ U2(π,π) ├──
     └─────────┘└───┘  └─────────┘  


## Running PyVOQC Directly

You can also call PyVOQC functions directly through the `VOQC` class in `pyvoqc.voqc`. However, we support limited operations over circuits (notably, we have do not allow printing circuits). But if all you need is QASM file input/output and gate counting, then this may be sufficient.

In [6]:
from pyvoqc.voqc import VOQC

# load circuit
c = VOQC("tutorial-files/tof_3_example.qasm")
print("Input circuit:")
c.print_info()

Input circuit:
Circuit uses 5 qubits and 3 gates.
{'CCX': 3}
No current layout.


In [7]:
# decompose CCX gates into single-qubit and CX (= cnot) gates
c.decompose_to_cnot()
print("After decomposing CCX gates:")
c.print_info()

After decomposing CCX gates:
Circuit uses 5 qubits and 45 gates.
{'H': 6, 'T': 12, 'Tdg': 9, 'CX': 18}
No current layout.


In [8]:
# run our most general optimization (see Sec. 4 of our POPL 2021 paper)
c.optimize_nam().replace_rzq()
print("After optimization:")
c.print_info()

After optimization:
Circuit uses 5 qubits and 40 gates.
{'H': 6, 'S': 2, 'T': 8, 'Sdg': 1, 'Tdg': 7, 'CX': 16}
No current layout.


In [9]:
# map the circuit to the Tenerife architecture with initial layout [0,1,2,3,4]
c.make_tenerife()
c.list_to_layout([0,1,2,3,4])
c.simple_map()
print("After mapping:")
c.print_info() # adds a bunch of CX and H gates

After mapping:
Circuit uses 5 qubits and 159 gates.
{'H': 98, 'S': 2, 'T': 8, 'Sdg': 1, 'Tdg': 7, 'CX': 43}
Current layout is [2,1,0,3,4]


In [10]:
# try optimizing again to remove introduced gates
c.cancel_single_qubit_gates().optimize_ibm()
print("\nAfter optimization (round 2):")
c.print_info()


After optimization (round 2):
Circuit uses 5 qubits and 113 gates.
{'U1': 2, 'U2': 56, 'U3': 12, 'CX': 43}
Current layout is [2,1,0,3,4]
